In [1]:
import numpy as np
import pandas as pd

data=pd.read_csv("tech_sort1k.csv")
# data.head(15)

data=data.drop(columns=["id","Note"])

data.head()

#replacing null vals with empty list
data['exact_matched_patt_contextual'] = [ [] if x is np.NaN else x for x in data['exact_matched_patt_contextual'] ]


import nltk
import re
from bs4 import BeautifulSoup

nltk.download("punkt")

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    return text


# for i, l in enumerate(data["exact_matched_patt_contextual"]):
#  print("list",i,"is",type(l))

In [5]:
#replacing null vals with empty list
data['exact_matched_patt_contextual'] = [ [] if x is np.NaN else x for x in data['exact_matched_patt_contextual'] ]


In [6]:
import nltk
import re
from bs4 import BeautifulSoup

nltk.download("punkt")

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    return text


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mohitbagaria/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
# for i, l in enumerate(data["exact_matched_patt_contextual"]):
#  print("list",i,"is",type(l))

In [8]:
data['exact_matched_patt_contextual']=data['exact_matched_patt_contextual'].astype(str)

In [9]:
# for i, l in enumerate(data["exact_matched_patt_contextual"]):
#  print("list",i,"is",type(l))

In [10]:
# data.head(15)

In [11]:
data["exact_matched_patt_contextual"] = data["exact_matched_patt_contextual"].apply(eval)

In [12]:
tech_dict = {}
for i in data['exact_matched_patt_contextual']:
    for j in i:
        if j not in tech_dict:
            tech_dict[j] = 1
        else:
            tech_dict[j] += 1

# tech_dict

tech_list=[]
tech_multi_words=[]
for i in tech_dict:
    if ((' ' in i) == True):
        tech_multi_words.append(i)
    else:
        tech_list.append(i)

In [13]:
tech_multi_words.append('microsoft power point')
tech_multi_words.append('sap pm')
tech_multi_words.append('sap erp')

In [14]:
len(tech_multi_words), len(set(tech_multi_words))

(77, 76)

In [15]:
print('pwc' in tech_list), 
print('ey' in tech_list), 
print('amazon' in tech_list),
print('flipkart' in tech_list),
print('microsoft' in tech_list),
print('sap' in tech_list),
print('ibm' in tech_list)

True
True
False
False
False
False
False


In [16]:
tech_list.append("amazon")
tech_list.append("flipkart")
tech_list.append("microsoft")
tech_list.append("sap")
tech_list.append("ibm")

In [17]:
# tech_multi_words[:5]

In [18]:
# tech_list[:5]

In [19]:
len(tech_list) , len(set(tech_list))

(319, 319)

In [20]:
data["summaries"]=data["summaries"].apply(clean_text)
data.summaries = data.summaries.replace('\s+', ' ', regex=True)

In [21]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches

In [22]:
def convert(lst):
    return ' '.join(lst).split()


def BIO_conversion(sentence):
    
    ind_dict = {}
    for i in tech_multi_words:
        if i in sentence:
#             ind_dict[sentence.index(i)]=i #word as val, key as index
            all_occ = list(find_all(sentence,i)) # [0, 5, 10, 15]
            for ind in all_occ:
                ind_dict[ind]=i

            
    temp_word=""
    final_word_sen=[]
    final_tag_sen = []
    start=-1
    end=-1
    for i in range(0, len(sentence)):
        if(i in range(start,end)):
#             print("break hua")
            continue
#         print(i)
        if sentence[i].isspace()==True and len(temp_word)>0:
            final_word_sen.append(temp_word)
            if temp_word in tech_list:
#                 print(temp_word)
                final_tag_sen.append("B")
            else:
                final_tag_sen.append("O")
            temp_word=""
        else: #not space
            try: #do we have multiword at this index
                temp_word=ind_dict[i]
#                 print("****")
                start=i+1
                end=i+len(temp_word) #here tag these multiwords and update new i
#                 print(i)
#                 print("****")
#                 print(temp_word)
                listofwords = convert([temp_word])
                f=True
                for word in listofwords:
                    final_word_sen.append(word)
                    if f:
                        final_tag_sen.append("B")
                        f=False
                    else:
                        final_tag_sen.append("I")
                temp_word=""
            except:
                if(sentence[i]!=' '):
                    temp_word+=sentence[i]
                
#     final_tag_sen.append(None) #adding blank
#     final_word_sen.append(None)#adding blank
#     print(len(sentence.split())-len(final_word_sen))
    return pd.DataFrame(list(zip(final_word_sen, final_tag_sen)),columns =['Word', 'Tag'])

In [23]:
# BIO_conversion(data["summaries"][18])

In [24]:
final_data=pd.DataFrame()

for i in range(0,len(data)):
      
    temp = BIO_conversion(data["summaries"][i])
    length = len(temp)
    wordd="Sentence :"+str(i+1) #sentence no.
    a=[wordd]*length
    temp.insert(0,"Sentence #",a)
    final_data = final_data.append(temp, ignore_index=True) #appending sentences in the required format

In [25]:
import seaborn as sns
sns.countplot(final_data['Tag'])

/Users/mohitbagaria/anaconda3/lib/python3.7/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


In [26]:
final_data['Tag'].value_counts()

O    320775
B      2303
I       312
Name: Tag, dtype: int64

In [27]:
df=final_data
df.loc[(df['Tag'] == 'B'), 'Tag'] = 'B-ORG'
df.loc[(df['Tag'] == 'I'), 'Tag'] = 'I-ORG'
df.to_csv("BIO_taggingdata_ALL.csv",index=False)

In [28]:
data["summaries"][12]

'bachelor of science bs degree in telecommunications networks and software program at politehnica university electronics telecommunications and information technology faculty master of science ms degree in management of networks and services program at politehnica university electronics telecommunications and information technology faculty routing and switching skills cisco ccna 640802 certification cisco ccnp route 642902 certification implementing cisco ip routing cisco ccnp switch 642813 certification implementing cisco ip switched networks cisco ccnp tshoot 642832 certification troubleshooting and maintaining cisco ip networks experience in troubleshooting and deploying networks using major l2 l3 protocols and platforms routing protocols rip ospf eigrp isis bgp data link protocols stp rstp mstp first hop redundancy protocols hsrp vrrp glbp mpls fundamentals cisco nexus juniper alcatel sr platforms unix linux basic master of science ms degree in management of networks and services p

In [29]:
# df=final_data2
# df.loc[(df['Tag'] == 'B'), 'Tag'] = 'B-ORG'
# df.loc[(df['Tag'] == 'I'), 'Tag'] = 'I-ORG'
# df.to_csv("BIO_taggingdata_B_MUST.csv",index=False)

In [30]:
data2 = pd.read_csv("5k_manual_tag_aman - Iteration_2_corrected sheet_with_30k_as_dictionary.csv")
# data2.head()

data2=data2.iloc[0:,0:3]

data2.columns=['id','summary','manually tagged']
# data2.head()

In [31]:
data.shape  , data2.shape

((1001, 2), (3633, 3))

In [32]:
data=pd.read_csv("tech_sort1k.csv")
data=data.drop(columns=["Note"])
data.head(10)
data['manually tagged']=data['exact_matched_patt_contextual']

data=data.drop(columns="exact_matched_patt_contextual")
data['manually tagged'] = [ [] if x is np.NaN else x for x in data['manually tagged'] ]


data.rename(columns={'summaries':'summary'}, inplace=True)
# data.head()

In [33]:


# newdf2 = data.append(data2, how="outer",on=['id','summary','manually tagged'])
newdf2 = pd.concat([data,data2])
newdf2=newdf2.drop_duplicates(subset='id', keep="last")
newdf2 = newdf2.reset_index(drop=True)

# for i, l in enumerate(newdf2["manually tagged"]):
#  print("list",i,"is",type(l))

In [34]:
newdf2['manually tagged'] = [ [] if x is np.NaN else x for x in newdf2['manually tagged'] ]
newdf2['manually tagged']=newdf2['manually tagged'].astype(str)

newdf2['manually tagged'] = newdf2['manually tagged'].apply(eval)

In [35]:
tech_dict = {}
for i in newdf2['manually tagged']:
    for j in i:
        if j not in tech_dict:
            tech_dict[j] = 1
        else:
            tech_dict[j] += 1

tech_list=[]
tech_multi_words=[]
for i in tech_dict:
    if ((' ' in i) == True):
        tech_multi_words.append(i)
    else:
        tech_list.append(i)

In [36]:
newdf2["summary"]=newdf2["summary"].apply(clean_text)

In [37]:
# final_data=pd.DataFrame()

# for i in range(0,len(data)):
#     temp = in_techwords(newdf2["summary"][i])
#     length = len(temp)
#     wordd="Sentence :"+str(i+1) #sentence no.
#     a=[wordd]*(length-1)
#     a.append(None)
#     temp.insert(0,"Sentence #",a)
#     final_data = final_data.append(temp, ignore_index=True) #appending sentences in the required format

In [38]:
final_data["Tag"].value_counts()

O        320775
B-ORG      2303
I-ORG       312
Name: Tag, dtype: int64

In [39]:
data_without_filter = final_data

In [40]:
final_data2=pd.DataFrame()

for i in range(0,len(data)):
    temp = BIO_conversion(newdf2["summary"][i])
    length = len(temp)
    wordd="Sentence :"+str(i+1) #sentence no.
    a=[wordd]*(length-1)
    a.append(None)
    temp.insert(0,"Sentence #",a)
#     if flag==True:
    final_data2 = final_data2.append(temp, ignore_index=True) #appending sentences in the required format

In [41]:
final_data2["Tag"].value_counts()

O    370577
B      4471
I       327
Name: Tag, dtype: int64

In [42]:
final_data2.to_csv("Manually_tagged_BIO.csv")